In [ ]:
# !pip install opencv-python mediapipe yagmail

   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/51.0 MB ? eta -:--:--
   - -------------------------------------- 2.1/51.0 MB 9.0 MB/s eta 0:00:06
   --------- ------------------------------ 12.3/51.0 MB 25.7 MB/s eta 0:00:02
   ------------- -------------------------- 17.0/51.0 MB 28.2 MB/s eta 0:00:02
   ------------------ --------------------- 23.6/51.0 MB 27.1 MB/s eta 0:00:02
   ----------------------------- ---------- 37.5/51.0 MB 34.5 MB/s eta 0:00:01
   ---------------------------------------  50.9/51.0 MB 41.0 MB/s eta 0:00:01
   ---------------------------------------- 51.0/51.0 MB 39.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 142.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/56.9 MB ? eta -:--:--
   ---------- ----------------------------- 14.9/56.9 MB 72.3 MB/s eta 0:00:01
   ----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ortools 9.11.4210 requires protobuf<5.27,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.


In [ ]:
# !pip install ultralytics

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------ --------- 0.8/1.0 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 5.6 MB/s eta 0:00:00


# Detection d'usage de telephone + envoie d'email avec capture de la preuve de cet incident

In [ ]:
import cv2
from ultralytics import YOLO
import yagmail # Pour envoyer des e-mails facilement depuis Python
import time

# Liste traduite des noms COCO
noms_classes_fr = [
    "personne", "vélo", "voiture", "moto", "avion", "bus", "train", "camion",
    "bateau", "feu de signalisation", "borne d'incendie", "panneau d'arrêt", "parcmètre", "banc",
    "oiseau", "chat", "chien", "cheval", "mouton", "vache", "éléphant", "ours", "zèbre", "girafe",
    "sac à dos", "parapluie", "sac à main", "cravate", "valise", "frisbee", "skis", "snowboard",
    "ballon de sport", "cerf-volant", "batte de baseball", "gant de baseball", "skateboard", "planche de surf",
    "raquette de tennis", "bouteille", "verre à vin", "tasse", "fourchette", "couteau", "cuillère", "bol", "banane",
    "pomme", "sandwich", "orange", "brocoli", "carotte", "hot-dog", "pizza", "beignet", "gâteau",
    "chaise", "canapé", "plante en pot", "lit", "table à manger", "toilettes", "téléviseur", "ordinateur portable", "souris",
    "télécommande", "clavier", "téléphone portable", "micro-ondes", "four", "grille-pain", "évier", "réfrigérateur",
    "livre", "horloge", "vase", "ciseaux", "ours en peluche", "sèche-cheveux", "brosse à dents"
]

# Chargement du modèle YOLOv8
modele = YOLO("yolov8l.pt")
modele.model.names = {i: nom for i, nom in enumerate(noms_classes_fr)}

# Initialisation de la webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640) # Largeur 
cap.set(4, 480) # Hauteur

# Configuration de l'envoi d'e-mail
EMAIL_SENDER = 'ka.ousmane@uam.edu.sn'
EMAIL_PASSWORD = 'ccba qcup mbwb jyra'
EMAIL_RECEIVER = 'ka.ousmane@uam.edu.sn'
yag = yagmail.SMTP(EMAIL_SENDER, EMAIL_PASSWORD)

# Variable de contrôle d'envoi pour éviter d’envoyer plusieurs e-mails pour la même détection
image_envoyee = False

# Vérifie si deux boîtes sont proches (personne et telephone portable)
def sont_proches(box1, box2, marge=50):
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2
    return not (x2 + marge < a1 or x1 - marge > a2 or y2 + marge < b1 or y1 - marge > b2)

print("Détection en cours... Appuyez sur Q pour quitter.")

SEUIL_TELEPHONE = 0.75   # seuil plus strict
TAILLE_MINI_OBJET = 40   # taille minimale pour considérer un objet

while True:
    success, frame = cap.read()
    if not success:
        print("Erreur de lecture du flux vidéo.")
        break

    resultats = modele(frame, stream=True, conf=0.5)

    for resultat in resultats:
        personnes = []
        telephones = []

        for box in resultat.boxes:
            cls_id = int(box.cls[0])
            nom_classe = noms_classes_fr[cls_id]
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            largeur = x2 - x1
            hauteur = y2 - y1

            # Filtrage selon les classes détectées
            if nom_classe == "personne":
                personnes.append((x1, y1, x2, y2))
                # Si la classe détectée est “personne”, on l’ajoute à la liste.

            elif nom_classe == "téléphone portable":
                if conf >= SEUIL_TELEPHONE and largeur > TAILLE_MINI_OBJET and hauteur > TAILLE_MINI_OBJET:
                    telephones.append((x1, y1, x2, y2))
                    # Si c’est un téléphone avec confiance suffisante et taille réaliste, on le garde aussi.
                else:
                    print(f"Ignoré {nom_classe} : conf={conf:.2f}, taille=({largeur},{hauteur})")

        # vérifie s’il y a une proximité spatiale entre une personne et un téléphone.
        for personne in personnes:
            for tel in telephones:
                if sont_proches(personne, tel) and not image_envoyee:
                    nom_fichier = f"capture_{int(time.time())}.jpg"
                    cv2.imwrite(nom_fichier, frame)
                    # enregistre l’image avec un nom unique basé sur l’heure actuelle
                    yag.send(
                        to=EMAIL_RECEIVER,
                        subject="Téléphone détecté avec une personne",
                        contents="Une personne a été détectée en train d'utiliser un téléphone.",
                        attachments=nom_fichier
                    )
                    print(f"Image envoyée à {EMAIL_RECEIVER} : {nom_fichier}")
                    image_envoyee = True

        image_annotée = resultat.plot()
        cv2.imshow("Détection - YOLOv8 (FR)", image_annotée)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Détection en cours... Appuyez sur Q pour quitter.

0: 480x640 1 personne, 1 chaise, 917.1ms
Speed: 9.3ms preprocess, 917.1ms inference, 8.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personne, 1 chaise, 1267.9ms
Speed: 11.0ms preprocess, 1267.9ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personne, 1036.9ms
Speed: 5.0ms preprocess, 1036.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personne, 853.0ms
Speed: 2.6ms preprocess, 853.0ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personne, 1 chaise, 841.7ms
Speed: 3.0ms preprocess, 841.7ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personne, 897.5ms
Speed: 3.6ms preprocess, 897.5ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personne, 1143.8ms
Speed: 2.7ms preprocess, 1143.8ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0:

KeyboardInterrupt: 

# Ajout de filtre avec lunette et chapeau en tenant compte de l'inclinaison

In [ ]:
import cv2
import mediapipe as mp
import math
import numpy as np

mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(refine_landmarks=True)
mp_draw = mp.solutions.drawing_utils  # Pour afficher les points détectés

# Chargement des images (avec transparence, canal alpha)
lunette = cv2.imread("lunette.png", cv2.IMREAD_UNCHANGED)
chapeau = cv2.imread("chapeau.png", cv2.IMREAD_UNCHANGED)

# Fonction pour coller une image PNG avec rotation et transparence sur l’image de la webcam
def superposer_image(image, filtre, centre, taille, angle_deg):
    # image = image du webcam
    # filtre = image du filtre (lunette ou chapeau)
    # centre = position du filtre (centre des yeux ou tête)
    # taille = (largeur, hauteur) du filtre
    # angle_deg = angle de rotation du filtre

    filtre = cv2.resize(filtre, taille) # Redimensionne le filtre selon la taille souhaitée
    M = cv2.getRotationMatrix2D((taille[0] // 2, taille[1] // 2), angle_deg, 1) # Rotation autour du centre
    filtre = cv2.warpAffine(filtre, M, taille, borderValue=(0, 0, 0, 0)) # Applique la rotation au filtre

    # séparation des canaux couleur + alpha
    b, g, r, a = cv2.split(filtre)
    masque = cv2.merge((a, a, a))  # utilisé comme masque de transparence
    couleur = cv2.merge((b, g, r)) # image sans alpha

    # Définition de la zone sur laquelle superposer
    x, y = centre[0] - taille[0] // 2, centre[1] - taille[1] // 2
    x1, y1 = max(x, 0), max(y, 0)
    x2, y2 = min(x + taille[0], image.shape[1]), min(y + taille[1], image.shape[0])

    roi = image[y1:y2, x1:x2] # ROI : zone de collage
    masque_crop = masque[y1 - y:y2 - y, x1 - x:x2 - x]
    couleur_crop = couleur[y1 - y:y2 - y, x1 - x:x2 - x]

    # Application du filtre sur le fond selon la transparence
    image[y1:y2, x1:x2] = np.where(masque_crop == 0, roi, couleur_crop)
    return image

# Calcule l’angle d’inclinaison latérale (gauche/droite) entre les yeux
def calcul_roll(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return math.degrees(math.atan2(dy, dx))

# Calcule l’inclinaison verticale (haut/bas) entre les yeux et le menton
def calcul_pitch(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return abs(math.degrees(math.atan2(dy, dx)))

# Activation de la webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break # Si aucun flux n’est lu, on quitte

    h, w = frame.shape[:2] # Dimensions de l’image
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Conversion en RGB pour MediaPipe
    resultats = face_mesh.process(rgb) # Détection des landmarks du visage

    if resultats.multi_face_landmarks:
        for visage in resultats.multi_face_landmarks:
            mp_draw.draw_landmarks(frame, visage, mp_face.FACEMESH_CONTOURS)

            # Points clés
            oeil_g = visage.landmark[33]
            oeil_d = visage.landmark[263]
            menton = visage.landmark[152]

            # Conversion des coordonnées en pixels
            xg, yg = int(oeil_g.x * w), int(oeil_g.y * h)
            xd, yd = int(oeil_d.x * w), int(oeil_d.y * h)
            xm, ym = int(menton.x * w), int(menton.y * h)

            centre_yeux = ((xg + xd) // 2, (yg + yd) // 2)

            # Distance visage verticale (yeux → menton)
            distance_face = ym - centre_yeux[1]

            # Placement lunettes - calcul de la largeur entre les yeux
            largeur_yeux = int(math.hypot(xd - xg, yd - yg))
            largeur_lunette = largeur_yeux * 2 # Taille des lunettes proportionnelle

            # Placement chapeau : au-dessus des yeux selon distance du visage
            chapeau_y = int(centre_yeux[1] - 1.2 * distance_face)
            decalage_x = int(roll * largeur_yeux * 0.02) # Ajustement horizontal du chapeau selon l’inclinaison gauche/droite
            # centre_chapeau = (centre_yeux[0], chapeau_y)
            centre_chapeau = (centre_yeux[0] + decalage_x, chapeau_y)
            largeur_chapeau = largeur_yeux * 3 # Taille du chapeau

            # Calcul des angles d’inclinaison
            roll = calcul_roll((xg, yg), (xd, yd))
            pitch = calcul_pitch((centre_yeux[0], centre_yeux[1]), (xm, ym))

            # Affichage de l’angle vertical
            cv2.putText(frame, f"Pitch: {pitch:.1f}°", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

            # Message en fonction de l'inclinaison détectée
            msg = "Tête alignée"
            if pitch < 80: msg = "Tête vers le BAS"
            elif pitch > 100: msg = "Tête vers le HAUT"
            cv2.putText(frame, msg, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (0, 255, 0) if "alignée" in msg else (0, 0, 255), 2)

            # Superposition des filtres dynamiques (lunettes et chapeau)
            frame = superposer_image(frame, lunette, centre_yeux, (largeur_lunette, largeur_yeux), -roll)
            frame = superposer_image(frame, chapeau, centre_chapeau, (largeur_chapeau, largeur_yeux * 2), -roll)
    
    # Affichage de l’image finale
    cv2.imshow("Chapeau parfait (position naturelle)", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
# Fermeture propre de la webcam
cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 